In [ ]:
import pandas as pd       
import requests        
from bs4 import BeautifulSoup         
import os      
import numpy as np     
import pandas as pd      
import yfinance as yf
from io import StringIO

In [2]:
def fetch_info():  
    try:      
        url = "https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average"
        headers = { 
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:101.0) Gecko/20100101 Firefox/101.0', 
            'Accept': 'application/json',
            'Accept-Language': 'en-US,en;q=0.5',
        }

                
        #  Send GET request
        response = requests.get(url, headers=headers)


        soup = BeautifulSoup(response.content, "html.parser")

        #  Get the symbols table
        tables = soup.find_all('table')
        #  #  Convert table to dataframe
        df = pd.read_html(StringIO(str(tables)))[2]
        #  Cleanup
        df.drop(columns=['Notes'], inplace=True)       
        return df    
    except Exception as e:   
        print(f'Error loading data  {e}')
        return None
    
dji_df = fetch_info()
print(dji_df)

               Company Exchange Symbol                        Industry  \
0                   3M     NYSE    MMM                    Conglomerate   
1     American Express     NYSE    AXP              Financial services   
2                Amgen   NASDAQ   AMGN               Biopharmaceutical   
3               Amazon   NASDAQ   AMZN                       Retailing   
4                Apple   NASDAQ   AAPL          Information technology   
5               Boeing     NYSE     BA           Aerospace and defense   
6          Caterpillar     NYSE    CAT         Construction and mining   
7              Chevron     NYSE    CVX              Petroleum industry   
8                Cisco   NASDAQ   CSCO          Information technology   
9            Coca-Cola     NYSE     KO                  Drink industry   
10              Disney     NYSE    DIS  Broadcasting and entertainment   
11                 Dow     NYSE    DOW               Chemical industry   
12       Goldman Sachs     NYSE     GS

### Tickers

In [4]:
tickers = dji_df.Symbol.values.tolist()
print(tickers)

['MMM', 'AXP', 'AMGN', 'AMZN', 'AAPL', 'BA', 'CAT', 'CVX', 'CSCO', 'KO', 'DIS', 'DOW', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'MCD', 'MRK', 'MSFT', 'NKE', 'PG', 'CRM', 'TRV', 'UNH', 'VZ', 'V', 'WMT']


### Download Tickers 

In [5]:
start_date = "2021-01-01"
end_date = "2022-09-01"
df = yf.download(tickers, start=start_date, end=end_date)
df.head

[*********************100%***********************]  30 of 30 completed


<bound method NDFrame.head of Price                       Adj Close                                      \
Ticker                           AAPL        AMGN        AMZN         AXP   
Date                                                                        
2021-01-04 00:00:00+00:00  126.683441  201.544525  159.331497  112.164032   
2021-01-05 00:00:00+00:00  128.249725  202.522629  160.925507  112.762665   
2021-01-06 00:00:00+00:00  123.932640  207.404297  156.919006  116.934113   
2021-01-07 00:00:00+00:00  128.161636  208.089020  158.108002  116.009209   
2021-01-08 00:00:00+00:00  129.267792  212.063675  159.134995  116.123611   
...                               ...         ...         ...         ...   
2022-08-25 00:00:00+00:00  168.166733  230.739594  137.279999  157.373291   
2022-08-26 00:00:00+00:00  161.826981  226.015503  130.750000  152.600601   
2022-08-29 00:00:00+00:00  159.611526  224.822739  129.789993  149.913498   
2022-08-30 00:00:00+00:00  157.168579  224.578

In [6]:
df.columns

MultiIndex([('Adj Close', 'AAPL'),
            ('Adj Close', 'AMGN'),
            ('Adj Close', 'AMZN'),
            ('Adj Close',  'AXP'),
            ('Adj Close',   'BA'),
            ('Adj Close',  'CAT'),
            ('Adj Close',  'CRM'),
            ('Adj Close', 'CSCO'),
            ('Adj Close',  'CVX'),
            ('Adj Close',  'DIS'),
            ...
            (   'Volume',  'MMM'),
            (   'Volume',  'MRK'),
            (   'Volume', 'MSFT'),
            (   'Volume',  'NKE'),
            (   'Volume',   'PG'),
            (   'Volume',  'TRV'),
            (   'Volume',  'UNH'),
            (   'Volume',    'V'),
            (   'Volume',   'VZ'),
            (   'Volume',  'WMT')],
           names=['Price', 'Ticker'], length=180)

In [7]:
df = df['Adj Close']
df.head

<bound method NDFrame.head of Ticker                           AAPL        AMGN        AMZN         AXP  \
Date                                                                        
2021-01-04 00:00:00+00:00  126.683441  201.544525  159.331497  112.164032   
2021-01-05 00:00:00+00:00  128.249725  202.522629  160.925507  112.762665   
2021-01-06 00:00:00+00:00  123.932640  207.404297  156.919006  116.934113   
2021-01-07 00:00:00+00:00  128.161636  208.089020  158.108002  116.009209   
2021-01-08 00:00:00+00:00  129.267792  212.063675  159.134995  116.123611   
...                               ...         ...         ...         ...   
2022-08-25 00:00:00+00:00  168.166733  230.739594  137.279999  157.373291   
2022-08-26 00:00:00+00:00  161.826981  226.015503  130.750000  152.600601   
2022-08-29 00:00:00+00:00  159.611526  224.822739  129.789993  149.913498   
2022-08-30 00:00:00+00:00  157.168579  224.578537  128.729996  150.029922   
2022-08-31 00:00:00+00:00  155.497147  225.686

### monthly return

In [9]:
mth_return_df = df.pct_change().resample("ME").agg(lambda x: (x+1).prod()-1)